#### BERT augmented CKY variant

Previously we looked at implememnting the CKY algorithm to generate parse trees for sentences conforming to a context-free grammar in Chomsky-normal-form. We found that sentences generated by a CFG suffer from syntax ambiguities, i.e. a sentence can have multiple valid parse trees, each with a different meaning. However, we know that out of the different possible parses, typically only one of them is the "correct" one, i.e. captures the intended meaning of the sentence. 

In this notebook, we will look an a variant of the CKY algorithm in which each possible span of a sentence is assigned a score, and these socres can be used to arrive at the correct parse tree. These scores can be computed with the help of a neural network trained on an annotated treebank dataset. A BERT model is powerful and well-suited for this task. The diagram below (borrowed from the Jurafsky-Martin textbook) summarizes the model architecture:

<img src="neural_parser.png" width="600" height="450">

We define the spans in the same way as we did for the vanilla CKY parser, i.e. using the "fencepost" positions. We also use the same upper-triangular matrix that which we used previously. Instead of using a pre-defined CFG in CNF to assign non-terminal labels to each element in this matrix (which represent the different possible spans), this time we will instead use BERT model to compute a distribution of scores over all possible terminals for each possible span. First, we create a fixed-size vector representation of the span and then feed it into an MLP classifier, as shown in the diagram. We outline the steps in more detail:

1) Convert words to subword tokens
2) Get `BERT embeddings for subwords`
3) Compute the `embeddings for full words` (many ways to do this, e.g. we could just assign the BERT em,bedding for the first subword of that word, or we could take the sum.average of the embedding of all the subwords or we could take element-wise max across all the subword embeddings, etc.)  
4) Compute `embeddings of fence posts` (shown as 0,1,2,3.. in the diagram above). Since each fence-post can represent the beginning or end of a span, we will create two separate representations. We first split the embedding vector $y_t$ of the $t$-th word in the sentence into two halves, $\overleftarrow{y_t}$ and $\overrightarrow{y_t}$ such that the concatentaion $[\overleftarrow{y_t}; \overrightarrow{y_t}] = y_t$. Then the `start-of-span representation` of the fencepost at position $i$ is defined as $\overrightarrow{y_i}$ and the `end-of-span representation` is defined as $\overleftarrow{y}_{i+1}$
5) Construct `embedding for a span` `(i,j)` using the fencepost embeddings as the following concatenation between the difference in start-of span and end-of-span embeddings for the bounding fenceposts: $v(i,j) = [\overrightarrow{y_j}-\overrightarrow{y_i}; \overleftarrow{y}_{j+1}-\overleftarrow{y}_{i+1}]$
6) Pass $v(i,j)$ through the MLP to get a distribution of scores over all possible non-terminal labels.